In [410]:
import pandas as pd
import pandas as pd

df = pd.read_csv('data/merged_data.csv').drop(['strat_9','strat_14','strat_5','strat_4'], axis=1).fillna(0)


In [416]:
# Given numerical values
values = [
    -0.023803622226665593, -0.017224230210854365, -0.04863676454429864, -0.04621342481308751,
    -0.04863676454429864, -0.01518631555797725, -0.03361237630287348, 0.04863676454429864,
    -0.018804204840069304, 0.03453552525824753, 0.03313319069573758, -0.04216647331014255,
    0.04863676454429864, -0.0076349630266866345, -0.04863676454429864, 0.04774373469261887,
    0.04863676454429864, 0.0459338857491553, 0.04863676454429864, 0.04863676454429864,
    -0.02907742252565288, 0.04863676454429864, 0.015724299788172528, 0.04863676454429864,
    0.023703132612985005, 0.03998268433623208, 0.013646076020177967, -0.025506792589677047
]

# Sum the values
sum_value = sum(abs(np.array(values)))
print(sum_value)


1.0000000000000002


## Linearity check

In [411]:
import pandas as pd
import numpy as np
from scipy.stats import linregress

def select_columns_with_high_r_squared(df_returns, r_squared_threshold=0.8):
    results = {}
    columns = []
    strat_slope_map = {}

    for column in df_returns.columns:
        x = np.arange(len(df_returns))
        y = df_returns[column].cumsum()
        slope, intercept, r_value, p_value, std_err = linregress(x, y)
        
        results[column] = {
            'slope': slope,
            'intercept': intercept,
            'r_squared': r_value**2,
            'p_value': p_value,
            'std_err': std_err
        }
        
        if r_value**2 > r_squared_threshold and p_value < .05:
            columns.append(column)
    
    for column in columns:
        slope_over_std = results[column]['slope'] / results[column]['std_err']
        strat_slope_map[column] = slope_over_std

    total_sum = sum(strat_slope_map.values())
    scaled_strat_slope_map = {strat: slope / total_sum for strat, slope in strat_slope_map.items()}

    return scaled_strat_slope_map

df_returns = df
output = select_columns_with_high_r_squared(df_returns)

c:\Users\elias\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2742: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]


In [412]:
import pandas as pd
import numpy as np
import quantstats as qs

def calculate_sharpe_ratio(df_returns, r_squared_threshold=0.8):
    output = select_columns_with_high_r_squared(df_returns, r_squared_threshold)
    scaled_slope_over_std_filtered = np.clip(list(output.values()), -0.1, 0.1)
    columns = list(output.keys())
    
    returns = np.dot(scaled_slope_over_std_filtered, df_returns[columns].values.T)
    
    sharpe_ratio = qs.stats.sharpe(pd.Series(returns))
    
    return sharpe_ratio

# Example usage:
sharpe_ratio = calculate_sharpe_ratio(df_returns)
print(sharpe_ratio)


1.784802430455841


In [ ]:
import numpy as np

def adjust_weights(weights, strat_names):
    lower_bound = -0.1
    upper_bound = 0.1
    weights = np.array(weights)
    adjusted_weights = np.where(weights > 0, upper_bound, lower_bound)
    current_sum = np.sum(adjusted_weights)
    scaling_factor = 1 / current_sum
    adjusted_weights *= scaling_factor
    
    adjusted_weights_dict = {strat_names[i]: adjusted_weights[i] for i in range(len(strat_names))}
    return adjusted_weights_dict

initial_weights = np.clip(list(output.values()), -0.1, 0.1)
strat_names = list(output.keys())  
adjusted_weights_dict = adjust_weights(initial_weights, strat_names)

print(adjusted_weights_dict)


{'strat_7': 0.05263157894736842, 'strat_8': 0.05263157894736842, 'strat_10': -0.05263157894736842, 'strat_19': 0.05263157894736842, 'strat_21': 0.05263157894736842, 'strat_24': 0.05263157894736842, 'strat_25': 0.05263157894736842, 'strat_26': 0.05263157894736842, 'strat_29': 0.05263157894736842, 'strat_30': 0.05263157894736842, 'strat_31': 0.05263157894736842, 'strat_32': 0.05263157894736842, 'strat_33': -0.05263157894736842, 'strat_34': 0.05263157894736842, 'strat_37': 0.05263157894736842, 'strat_38': 0.05263157894736842, 'strat_39': -0.05263157894736842, 'strat_40': 0.05263157894736842, 'strat_41': 0.05263157894736842, 'strat_42': 0.05263157894736842, 'strat_43': 0.05263157894736842, 'strat_45': 0.05263157894736842, 'strat_46': 0.05263157894736842, 'strat_48': 0.05263157894736842, 'strat_49': 0.05263157894736842}


In [414]:
def generate_team_info_from_scaled_weights(scaled_weights, team_name, passcode):
    strat_dict = {key: weight for i, (key, weight) in enumerate(scaled_weights.items())}
    strat_dict['team_name'] = team_name
    strat_dict['passcode'] = passcode
    return strat_dict

weights = adjusted_weights_dict
team_name = 'Longer Term Capital Management'
passcode = 'DogCat'

output = generate_team_info_from_scaled_weights(weights, team_name, passcode)
output

{'strat_7': 0.05263157894736842,
 'strat_8': 0.05263157894736842,
 'strat_10': -0.05263157894736842,
 'strat_19': 0.05263157894736842,
 'strat_21': 0.05263157894736842,
 'strat_24': 0.05263157894736842,
 'strat_25': 0.05263157894736842,
 'strat_26': 0.05263157894736842,
 'strat_29': 0.05263157894736842,
 'strat_30': 0.05263157894736842,
 'strat_31': 0.05263157894736842,
 'strat_32': 0.05263157894736842,
 'strat_33': -0.05263157894736842,
 'strat_34': 0.05263157894736842,
 'strat_37': 0.05263157894736842,
 'strat_38': 0.05263157894736842,
 'strat_39': -0.05263157894736842,
 'strat_40': 0.05263157894736842,
 'strat_41': 0.05263157894736842,
 'strat_42': 0.05263157894736842,
 'strat_43': 0.05263157894736842,
 'strat_45': 0.05263157894736842,
 'strat_46': 0.05263157894736842,
 'strat_48': 0.05263157894736842,
 'strat_49': 0.05263157894736842,
 'team_name': 'Longer Term Capital Management',
 'passcode': 'DogCat'}